# Classificação com o modelo Bernoulli de Naive Bayes

#### Temos um DataSet com dados de diversos clientes e suas caracteristicas. O objetivo é descobrir, com base neses dados, se um novo cliente provavelmente irá de desligar ou não dos serviços da empresa.

In [3]:
import pandas as pd
dados = pd.read_csv('/content/sample_data/Customer-Churn.csv')

In [5]:
dados.head()

,Maior65Anos,Conjuge,Dependentes,MesesDeContrato,TelefoneFixo,VariasLinhasTelefonicas,ServicoDeInternet,SegurancaOnline,BackupOnline,SeguroNoDispositivo,SuporteTecnico,TVaCabo,StreamingDeFilmes,TipoDeContrato,PagamentoOnline,FormaDePagamento,ContaMensal,Churn
0,0,Sim,Nao,1,Nao,SemServicoTelefonico,DSL,Nao,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,29.85,Nao
1,0,Nao,Nao,34,Sim,Nao,DSL,Sim,Nao,Sim,Nao,Nao,Nao,UmAno,Nao,ChequePapel,56.95,Nao
2,0,Nao,Nao,2,Sim,Nao,DSL,Sim,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequePapel,53.85,Sim
3,0,Nao,Nao,45,Nao,SemServicoTelefonico,DSL,Sim,Nao,Sim,Sim,Nao,Nao,UmAno,Nao,DebitoEmConta,42.30,Nao
4,0,Nao,Nao,2,Sim,Nao,FibraOptica,Nao,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,70.70,Sim


### Modificando os dados manualmente
#### 'Sim': 1, 
#### 'Nao': 0

In [6]:
traducao_dic = {'Sim': 1, 
                'Nao': 0}

dadosmodificados = dados[['Conjuge', 'Dependentes', 'TelefoneFixo', 'PagamentoOnline', 'Churn']].replace(traducao_dic)
dadosmodificados.head()

,Conjuge,Dependentes,TelefoneFixo,PagamentoOnline,Churn
0,1,0,0,1,0
1,0,0,1,0,0
2,0,0,1,1,1
3,0,0,0,0,0
4,0,0,1,1,1


### Modificando os dados com get_dummies

In [7]:
dummie_dados = pd.get_dummies(dados.drop(['Conjuge', 'Dependentes', 'TelefoneFixo', 'PagamentoOnline', 'Churn'],
                axis=1))

#junção dos dados trasformados com os que já tinhamos
dados_final = pd.concat([dadosmodificados, dummie_dados], axis=1)

In [8]:
dados_final.head()

,Conjuge,Dependentes,TelefoneFixo,PagamentoOnline,Churn,Maior65Anos,MesesDeContrato,ContaMensal,VariasLinhasTelefonicas_Nao,VariasLinhasTelefonicas_SemServicoTelefonico,...,StreamingDeFilmes_Nao,StreamingDeFilmes_SemServicoDeInternet,StreamingDeFilmes_Sim,TipoDeContrato_DoisAnos,TipoDeContrato_Mensalmente,TipoDeContrato_UmAno,FormaDePagamento_CartaoDeCredito,FormaDePagamento_ChequeDigital,FormaDePagamento_ChequePapel,FormaDePagamento_DebitoEmConta
0,1,0,0,1,0,0,1,29.85,0,1,...,1,0,0,0,1,0,0,1,0,0
1,0,0,1,0,0,0,34,56.95,1,0,...,1,0,0,0,0,1,0,0,1,0
2,0,0,1,1,1,0,2,53.85,1,0,...,1,0,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,45,42.30,0,1,...,1,0,0,0,0,1,0,0,0,1
4,0,0,1,1,1,0,2,70.70,1,0,...,1,0,0,0,1,0,0,1,0,0


### Separando as variaveis X e Y

In [24]:
X = dados_final.drop('Churn', axis = 1)
y = dados_final['Churn']

## Utilizando o StandardScaler para ajustar as Escalas

In [ ]:
from sklearn.preprocessing import StandardScaler

norm = StandardScaler()

X_normalizado = norm.fit_transform(X)

### Separando Treino e teste

In [37]:
from sklearn.model_selection import train_test_split

X_treino, X_teste, y_treino, y_teste = train_test_split(X_normalizado, y, test_size=0.3, random_state=123)

### Instanciando modelo

In [43]:
from sklearn.naive_bayes import BernoulliNB

In [44]:
bnb = BernoulliNB(binarize=-0.44)

In [45]:
#escolho utilizar mediana, porque é o valor central dos nossos dados ordenados
np.median(X_treino)

-0.4461759755508453

#### binarize=-0.44

Se tiver acima de 0.44, classifica como 1, se tiver abaixo, classifica como 0..


Usado para transformar características não-binárias em binárias. 


O modelo so usa caracteristicas binarias, por isso a transformação é necessaria

## Fazendo o fit e predição do modelo

In [48]:
bnb.fit(X_treino, y_treino)

BernoulliNB(binarize=-0.44)

In [49]:
predito_BNb = bnb.predict(X_teste)

## Avaliando o modelo

### Matriz de confusão

In [100]:
from sklearn.metrics import confusion_matrix

In [102]:
print(confusion_matrix(y_teste, predito_BNb))

[[1050  519]
 [ 242 1294]]


###  Acurácia


In [104]:
from sklearn.metrics import accuracy_score

In [106]:
#modelo Bernoulli de naive bayes
print(accuracy_score(y_teste, predito_BNb))

0.7549114331723027


### Precisão

In [108]:
from sklearn.metrics import precision_score

In [110]:
#modelo Bernoulli de naive bayes
print(precision_score(y_teste, predito_BNb))

0.7137341423055709


### Recall

In [112]:
from sklearn.metrics import recall_score

In [114]:
#modelo Bernoulli de naive bayes
print(recall_score(y_teste, predito_BNb))

0.8424479166666666
